In [10]:
import ipywidgets as widgets
from IPython.display import display
from ultralytics import YOLO
from PIL import Image
import io
import numpy as np

# ================= CẤU HÌNH =================
MODEL_PATH = 'taco1-v12s.pt' 
CONF_THRESHOLD = 0.5  # Độ tin cậy (thấp thì bắt nhạy, cao thì bắt chắc)
# ============================================

def start_testing_app():
    # 1. Tải model
    print(f"⏳ Đang tải model từ: {MODEL_PATH} ...")
    try:
        model = YOLO(MODEL_PATH)
        print("✅ Model đã sẵn sàng! Hãy bấm nút bên dưới để chọn ảnh.")
    except Exception as e:
        print(f"❌ Lỗi không tìm thấy model: {e}")
        return

    # 2. Tạo nút Upload
    uploader = widgets.FileUpload(
        accept='image/*',  # Chỉ chấp nhận file ảnh
        multiple=False,    # Mỗi lần test 1 ảnh
        description='📂 Chọn ảnh Test',
        button_style='primary' # Màu xanh cho đẹp
    )

    # 3. Vùng hiển thị kết quả
    out = widgets.Output()

    # 4. Hàm xử lý khi người dùng chọn ảnh
    def on_upload_change(change):
        if not change['new']:
            return
            
        with out:
            out.clear_output() # Xóa kết quả cũ
            try:
                print("🔄 Đang xử lý...")
                
                # Lấy dữ liệu file ảnh từ widget (xử lý sự khác biệt phiên bản widget)
                if isinstance(change['new'], dict): # Bản cũ
                    item = next(iter(change['new'].values()))
                    content = item['content']
                elif isinstance(change['new'], tuple) or isinstance(change['new'], list): # Bản mới
                    item = change['new'][0]
                    content = item['content']
                else:
                    item = change['new']
                    content = item['content']

                # Chuyển bytes thành ảnh
                img_pil = Image.open(io.BytesIO(content))
                
                # Chạy dự đoán
                results = model.predict(img_pil, conf=CONF_THRESHOLD)
                
                # Lấy kết quả vẽ khung (Plot)
                # Lưu ý: YOLO trả về BGR (chuẩn OpenCV), cần đổi sang RGB để hiển thị đúng màu
                res_array = results[0].plot() 
                res_image = Image.fromarray(res_array[..., ::-1]) 

                # Hiển thị
                print(f"🎉 Kết quả: Tìm thấy {len(results[0].boxes)} vật thể.")
                display(res_image)
                
            except Exception as e:
                print(f"❌ Có lỗi xảy ra khi đọc ảnh: {e}")
                # Reset uploader để chọn lại ảnh khác nếu lỗi
                uploader.value = ()

    # Gắn sự kiện vào nút upload
    uploader.observe(on_upload_change, names='value')
    
    # Hiển thị ra màn hình
    display(uploader, out)

# --- CHẠY HÀM ---
start_testing_app()




⏳ Đang tải model từ: taco1-v12s.pt ...
✅ Model đã sẵn sàng! Hãy bấm nút bên dưới để chọn ảnh.


FileUpload(value=(), accept='image/*', button_style='primary', description='📂 Chọn ảnh Test')

Output()